In [15]:
%pip install llama-index-llms-ollama

Note: you may need to restart the kernel to use updated packages.


In [16]:
!pip install llama-index pypdf

In [17]:
from llama_index.llms.ollama import Ollama
llm = Ollama(model="llama2:7b", request_timeout=3000)

In [18]:
llm.complete("Hello")

INFO:httpx:HTTP Request: POST http://localhost:11434/api/generate "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/generate "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/generate "HTTP/1.1 200 OK"


CompletionResponse(text='Hi there! How are you today?', additional_kwargs={'model': 'llama2:7b', 'created_at': '2024-02-16T15:08:08.4482002Z', 'done': True, 'context': [518, 25580, 29962, 3532, 14816, 29903, 29958, 5299, 829, 14816, 29903, 6778, 13, 13, 10994, 518, 29914, 25580, 29962, 13, 18567, 727, 29991, 1128, 526, 366, 9826, 29973], 'total_duration': 2098611300, 'load_duration': 5682600, 'prompt_eval_count': 14, 'prompt_eval_duration': 1206253000, 'eval_count': 9, 'eval_duration': 885361000}, raw={'model': 'llama2:7b', 'created_at': '2024-02-16T15:08:08.4482002Z', 'response': 'Hi there! How are you today?', 'done': True, 'context': [518, 25580, 29962, 3532, 14816, 29903, 29958, 5299, 829, 14816, 29903, 6778, 13, 13, 10994, 518, 29914, 25580, 29962, 13, 18567, 727, 29991, 1128, 526, 366, 9826, 29973], 'total_duration': 2098611300, 'load_duration': 5682600, 'prompt_eval_count': 14, 'prompt_eval_duration': 1206253000, 'eval_count': 9, 'eval_duration': 885361000}, delta=None)

# Create service context

# Indexing

In [19]:
!pip install llama-index-embeddings-huggingface

In [20]:
from llama_index.core.callbacks import LlamaDebugHandler
from llama_index.core.callbacks import CallbackManager
from llama_index.core import SimpleDirectoryReader, ServiceContext, set_global_service_context, StorageContext, \
    VectorStoreIndex

documents = (
    SimpleDirectoryReader(
        input_dir = './data',
        required_exts = [".pdf"])
        .load_data()
)

llama_debug = LlamaDebugHandler(print_trace_on_end=True)
callback_manager = CallbackManager([llama_debug])

service_context = (
    ServiceContext
    .from_defaults(
        llm=llm, 
        embed_model="local:BAAI/bge-small-en-v1.5", 
        chunk_size=300,
        callback_manager=callback_manager
    )
)
set_global_service_context(service_context)

nodes = (
    service_context
    .node_parser
    .get_nodes_from_documents(documents)
)

storage_context = StorageContext.from_defaults()
storage_context.docstore.add_documents(nodes)

index = (
    VectorStoreIndex
    .from_documents(
        documents, 
        storage_context=storage_context, 
        llm=llm
        )
)

ERROR:pypdf._cmap:Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
ERROR:pypdf._cmap:Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
ERROR:pypdf._cmap:Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
ERROR:pypdf._cmap:Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
ERROR:pypdf._cmap:Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
ERROR:pypdf._cmap:Advanced encoding /SymbolSetEncoding 

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_1604\4238855961.py:18: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  .from_defaults(


**********
Trace: index_construction
    |_CBEventType.EMBEDDING ->  2.357857 seconds
    |_CBEventType.EMBEDDING ->  1.832631 seconds
    |_CBEventType.EMBEDDING ->  1.802971 seconds
    |_CBEventType.EMBEDDING ->  2.095322 seconds
    |_CBEventType.EMBEDDING ->  1.699826 seconds
    |_CBEventType.EMBEDDING ->  1.901398 seconds
    |_CBEventType.EMBEDDING ->  1.635072 seconds
    |_CBEventType.EMBEDDING ->  1.72603 seconds
    |_CBEventType.EMBEDDING ->  1.615471 seconds
    |_CBEventType.EMBEDDING ->  1.882479 seconds
    |_CBEventType.EMBEDDING ->  1.717634 seconds
    |_CBEventType.EMBEDDING ->  1.852808 seconds
    |_CBEventType.EMBEDDING ->  1.78836 seconds
    |_CBEventType.EMBEDDING ->  2.785124 seconds
    |_CBEventType.EMBEDDING ->  2.245576 seconds
    |_CBEventType.EMBEDDING ->  1.916762 seconds
    |_CBEventType.EMBEDDING ->  1.714477 seconds
    |_CBEventType.EMBEDDING ->  1.691546 seconds
    |_CBEventType.EMBEDDING ->  1.671815 seconds
    |_CBEventType.EMBEDDING ->  1.

In [21]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

# Creating an engine of llm
Query engine has several types of use: simple querying, chat mode, stream chat. 
All of those can be used asynchronously. 

In [22]:
query_engine = index.as_query_engine()

In [23]:
response = query_engine.query("Give me unit structure of Introduction to big data")
print(response)

INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
**********
Trace: query
    |_CBEventType.QUERY ->  63.902809 seconds
      |_CBEventType.RETRIEVE ->  0.05602 seconds
        |_CBEventType.EMBEDDING ->  0.033817 seconds
      |_CBEventType.SYNTHESIZE ->  63.84627 seconds
        |_CBEventType.TEMPLATING ->  0.0 seconds
        |_CBEventType.LLM ->  63.83163 seconds
**********
Unit Structure:

1.0 Objectives

* Introduce the concept of Big Data
* Discuss the characteristics of Data and Big Data
* Provide an overview of the evolution of Big Data
* Define Big Data and its challenges
* Explain why Big Data is important
* Describe the data warehouse environment and the difference between traditional business intelligence and Big Data analytics
* Outline ke

In [24]:
response = query_engine.query("What is main objective of Big Data")
print(response)

INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
**********
Trace: query
    |_CBEventType.QUERY ->  40.348925 seconds
      |_CBEventType.RETRIEVE ->  0.091674 seconds
        |_CBEventType.EMBEDDING ->  0.078637 seconds
      |_CBEventType.SYNTHESIZE ->  40.257251 seconds
        |_CBEventType.TEMPLATING ->  0.0 seconds
        |_CBEventType.LLM ->  40.252306 seconds
**********
The main objective of Big Data is to derive new value and create competitive advantage for organizations by leveraging their most valuable asset - information.


In [25]:
response = query_engine.query("What is Key roles of the new big data ecosystems")
print(response)

INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
**********
Trace: query
    |_CBEventType.QUERY ->  62.575967 seconds
      |_CBEventType.RETRIEVE ->  0.081366 seconds
        |_CBEventType.EMBEDDING ->  0.064068 seconds
      |_CBEventType.SYNTHESIZE ->  62.494601 seconds
        |_CBEventType.TEMPLATING ->  0.0 seconds
        |_CBEventType.LLM ->  62.490369 seconds
**********
Based on the provided context information, the key roles for the New Big Data Ecosystem are:

1. Deep Analytical Talent: This group consists of technically savvy individuals with strong analytical skills. They possess a combination of skills to handle raw, unstructured data and apply complex analytical techniques at massive scales. Examples of current professions fitting into 

In [26]:
# Building 